### **LangGraph**

**State** (an object) aka short term memory: The State typically defines the data structure that will be passed between nodes in the graph. It can be understood as the shared memory / context that each step of the workflow can read from and write to.

**Workflow**: StateGraph workflow is a powerful way to orchestrate multi-step processes

#### Simple single-node graph

In [ ]:
# Define State's pydantic model / shcema
from pydantic import BaseModel

class State(BaseModel):
    message: str
    answer: str=""
    vibe: str


In [ ]:
# A node is an action (a function that takes in a state and returns a dict)
def append_vibes_to_query(state: State) -> dict:
    """Generate value for state key 'answer' by appending the vibe to the message"""
    return {"answer": f"{state.message} {state.vibe}"}    

In [ ]:
# Create a simple graph - workflow (for now empty)
from langgraph.graph import StateGraph, START, END

#  Initializes a new graph-based workflow
workflow = StateGraph(State)

# Define nodes (functions)
workflow.add_node("append_vibes_to_query", append_vibes_to_query)

# Add edges (add START-entry point and END nodes)
# START is a special constant indicating the beginning of the graph. 
workflow.add_edge(START, "append_vibes_to_query")
# END is a special constant indicating the end of the graph. 
workflow.add_edge("append_vibes_to_query", END)

# compiles the defined workflow into an executable graph
# Run the graph
graph = workflow.compile()

In [ ]:
# Display the graph
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Initiate th state
initial_state = {
    "message": "Give me some vibes!", 
    "vibe": "I'm feeling like a badass today!"
    }

# Invoke the graph (compute the state, the results is a dict of the from we defined in pydantic model)
result = graph.invoke(initial_state)

# Display the result - we evolved the state with adding valua to a key "answer"
result

#### Conditional graph (a base for router)

In [ ]:
# Define pydantic model for state
from pydantic import BaseModel

class State(BaseModel):
    message: str
    answer: str=""

In [ ]:
# node
def append_vibes_to_query(state: State) -> dict:
    """Generate value for state key 'answer' by overwriting the existing value"""
    return {"answer": "I am here to add some vibes."}


In [ ]:
# another node (router)
from typing import Literal
import random
def router(state: State) -> Literal["append_vibe_1", "append_vibe_2", "append_vibe_3"]:
    """Route the state to the appropriate node"""
    vibes = ["append_vibe_1", "append_vibe_2", "append_vibe_3"]
    vibe_path = random.choice(vibes)
    return vibe_path

In [ ]:
# node to route to
# appends vibe to the existing answer
def append_vibe_1(state: State) -> dict:
    vibe = "I'm feeling like a badass today!"
    return {"answer": f"{state.answer} {state.message} {vibe}"}

def append_vibe_2(state: State) -> dict:
    vibe = "I'm feeling chill today!"
    return {"answer": f"{state.answer} {state.message} {vibe}"}

def append_vibe_3(state: State) -> dict:
    vibe = "I'm sick today!"
    return {"answer": f"{state.answer} {state.message} {vibe}"}

In [ ]:
# grpah
workflow = StateGraph(State)

# a node (action or computation)
workflow.add_node("append_vibes_to_query", append_vibes_to_query)
workflow.add_node("append_vibe_1", append_vibe_1)
workflow.add_node("append_vibe_2", append_vibe_2)
workflow.add_node("append_vibe_3", append_vibe_3)

# edges (from - to)
# Start with append_vibes_to_query
workflow.add_edge(START, "append_vibes_to_query")
# Route to the appropriate node
workflow.add_conditional_edges("append_vibes_to_query", router)
# End with the appropriate node
workflow.add_edge("append_vibe_1", END)
workflow.add_edge("append_vibe_2", END)
workflow.add_edge("append_vibe_3", END)

# Run the graph
graph = workflow.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Run the graph
# Initiate th state
initial_state = {"message": "Give me some vibes!"}
result = graph.invoke(initial_state)
result

---

### **Agent graph (with LLM decision)**

In [ ]:
initial_state = {"message": "Give me some vibes!"}

In [ ]:
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

from langchain_core.messages import AIMessage, ToolMessage

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import random
import ast
import inspect
import instructor
import json

A tool

In [ ]:
# A tool to be used in a graph
# Write properly with type hints and description to be used as a tool by LLM
def append_vibes(query: str, vibe: str) -> str:
    """Takes in a query and a vibe and returns a string with the query and vibe appended.

    Args:
        query: The query to append the vibe to.
        vibe: The vibe to append to the query.

    Returns:
        A string with the query and vibe appended.
    """

    print(f"{query} {vibe}")
    return f"{query} {vibe}"

In [ ]:
# # ---------------------------------------------------------------------------
# # Explanation
# #
# # Prepare the function to be used in a graph by LLM
# # 1. stringify the function 
# # 2. parse the function to extract values (json schema format)
# # 3. inject the description to the prompt

# print("Stringify the function")
# # inspect.getsource() with globals() vs without
# # globals() approach is more flexible (looks up for a string in globals, not necessarily the function object).
# function_string = inspect.getsource(globals()["append_vibes"])
# # function_string = inspect.getsource(append_vibes)
# print(function_string)

# print("--------------------------------")
# print("Parse the function")
# # Parse the function (at the bottom of the notebook)
# result = parse_function_definition(function_string)
# result
# # # ---------------------------------------------------------------------------
# # Why This Structure?
# # This follows the JSON Schema format, which is commonly used for API documentation and validation:
# # parameters represents the overall parameter schema
# # properties contains the individual parameter definitions
# # This structure allows for easy validation and documentation generation

Pydantic models

In [ ]:
# Define a state schema (a bit more complex)
from typing import Annotated, List, Dict, Any
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph
from operator import add

class ToolCall(BaseModel):
    name: str  # Eg.: append_vibes
    arguments: dict  # Eg.: {"query": "How are you?", "vibe": "Feeling good"}

class AgentResponse(BaseModel):
    answer: str
    tool_calls: List[ToolCall] = Field(default_factory=list)

# add is a reducer, it means that if any node is returning dict or other data object 
# with a key "messages", it will be added to the messages list
# keys that do not have add, will be overwritten
class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    message: str = ""
    iteration: int = Field(default=0)
    answer: str = ""
    available_tools: List[Dict[str, Any]] = []
    tool_calls: List[ToolCall] = Field(default_factory=list) # ToolCall is a pydantic model


Nodes: LLM node (agent node)

In [ ]:
# Agent node
from jinja2 import Template
from langchain_core.messages import AIMessage, ToolMessage
import instructor
from openai import OpenAI


# tojson  >>> converts dict to json string
# - Once you have the vibe from the tool, return it as an answer, you can reformulate it a bit.

def agent_node(state: State) -> dict:

   prompt_template =  """You are a assistant that is generating vibes for a user.

You will be given a selection of tools you can use to add vibes to a user's query.

<Available tools>
{{ available_tools | tojson }}
</Available tools>

When you need to use a tool, format your response as:

<tool_call>
{"name": "tool_name", "arguments": {...}}
</tool_call>

Instructions:
- You need to use the tools to add vibes to the user's query.
- Add a random vibe to the user's query.
"""
   # 1. Make a template of the prompt
   # Jinja2 template to inject the available_tools to the prompt
   template = Template(prompt_template)
   
   # 2. Inject the available_tools to the prompt
   # available_tools is a list of tool descriptions & be passed to the prompt {{ }}
   # In prompt it will be rendered to a json string
   # tolls are extracted from tool_descriptions = get_tool_descriptions_from_node(tool_node)
   prompt = template.render(available_tools=state.available_tools)

   # 3. Use the instructor to enforce structured output; initialize the client
   # The instructor.from_openai() function "patches" or wraps the standard OpenAI client. 
   # This patching process enhances the client with Instructor's capabilities, primarily 
   # its ability to enforce structured output from large language models (LLMs).
   client = instructor.from_openai(OpenAI())

   # 4. Use the client to create a response
   # response is a pydantic model, AgentResponse
   # raw_response is a raw response from the LLM
   response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        response_model=AgentResponse,
        messages=[
           {"role": "system", "content": prompt}, # promt with available tools
           {"role": "user", "content": state.message}], # initial message, or user message
        temperature=0.5,
        #temperature=1.5,
   )

   # 5. Create an ai message (AIMessage is a langchain message type; like open AI has "user", "system", "tool")
   # IF: If there are tool calls, collect tool calls & their arguments in a list
   # ELSE: If there are no tool calls, just return the regular LLM response
   if response.tool_calls:
      tool_calls = []
      for i, tc in enumerate(response.tool_calls): # collect tool call info
         tool_calls.append({
               "id": f"call_{i}",
               "name": tc.name,
               "args": tc.arguments
         })
      ai_message = AIMessage( 
         content=response.answer, # answer is a string, regular LLM response
         tool_calls=tool_calls    # tool calls is a list of tool calls & their arguments
         )
   else: 
      ai_message = AIMessage( 
         content=response.answer, # just the answer, no tool calls
      )

   return {
      "messages": [ai_message],
      "tool_calls": response.tool_calls
   }
   

Node: router (tools selector)

In [ ]:
# If a state has tools to be executed, we go to the tool node
# If not, we end the workflow
def tool_router(state: State) -> str:
    """Decide whether to continue or end"""
    
    if len(state.tool_calls) > 0:
        return "tools"
    else:
        return "end"

A graph

In [ ]:
from langgraph.prebuilt import ToolNode

# 1. Initialize a graph
# Buil a graph with a tool. Graph name is "workflow"
# This initializes a new graph-based workflow. 
workflow = StateGraph(State)

# 2. Prepare the tools and info about them (tool_descriptions)
# Available tools & dedicated langgraph node that executes tools
tools = [append_vibes]
# tool node will try to envoke / execute the tool
tool_node = ToolNode(tools)
display(tool_node)

# ToolNode has all the tools, we exctract descriptions from it
# We use wrapper function to extract the tool description, it is defined at the bottom of the notebook
# Wrapper function loops through the tools and extracts the tool description usin parse_function_definition
# Returns a list of tool descriptions; which will be used to inject to the State at the start as initial_state
tool_descriptions = get_tool_descriptions_from_node(tool_node)
display(tool_descriptions)

# 3. Add nodes to the graph
# Nodes
# Agent node is a function defined above
workflow.add_node("agent_node", agent_node)
workflow.add_node("tool_node", tool_node)

# 4. Add edges to the graph
# Edges
workflow.add_edge(START, "agent_node")
# Conditional edge; 
workflow.add_conditional_edges(
    "agent_node", # from agent node
    tool_router,  # use router to decide where to go next
    {"tools": "tool_node",  # true branch (execute tools)
     "end": END}            # false branch
)

graph = workflow.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
tool_descriptions

In [ ]:
# 1. Invoke the graph with the initial state, which has the message & available tools
initial_state = {
    "message": "Give me some vibes!",
    "available_tools": tool_descriptions
}

# # More proper way to define the initial state, as pydantic model expects
# initial_state = State(
#     **initial_state
# )

# # or
# initial_state = State(
#     message="Give me some vibes!",
#     available_tools=tool_descriptions
# )

# LLM cames up with the answer; 
# the warning is because the graph expects State object, not simple dict
# Here's what's happening:
# agent_node - Makes the LLM decision and creates tool calls
# tool_node - Executes the append_vibes function
# append_vibes function - This is where the print happens (answer is printed)
result = graph.invoke(initial_state)

In [ ]:
result

In [ ]:
for data in result["messages"]:
    print(data)

### **Agent graph with Loopback from Tools**
React patern (reason and act: loops) - 1:10

React agent implemented without LangChain, but using LangGraph and Instructor

https://maven.com/swirl-ai/end-to-end-ai-engineering/1/syllabus/modules/3d5e45?item=9afl7815qpu
https://github.com/swirl-ai/sprint-03-ai-engineering-bootcamp/blob/main/notebooks/05-LangGraph-intro.ipynb 

In [ ]:
# Pydantic models (for structured outputs)

from typing import Annotated, List, Dict, Any
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph
from operator import add

# ToolCalls
class ToolCall(BaseModel):
    name: str  # Eg.: append_vibes
    arguments: dict  # Eg.: {"query": "How are you?", "vibe": "Feeling good"}

# AgentResponse
class AgentResponse(BaseModel):
    answer: str
    tool_calls: List[ToolCall] = Field(default_factory=list)

# State (short term memory: all messages and tool calls)
# NEW: stop condion (we will use iteration) it was before, just highlithing
class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    message: str = ""
    iteration: int = Field(default=0)
    answer: str = ""
    available_tools: List[Dict[str, Any]] = []
    tool_calls: List[ToolCall] = Field(default_factory=list)

In [ ]:
# A tool (without print)
def append_vibes(query: str, vibe: str) -> str:
    """Takes in a query and a vibe and returns a string with the query and vibe appended.

    Args:
        query: The query to append the vibe to.
        vibe: The vibe to append to the query.

    Returns:
        A string with the query and vibe appended.
    """
    return f"{query} {vibe}"

In [ ]:
# Agent (with short term memory)

from jinja2 import Template
from langchain_core.messages import AIMessage, ToolMessage
import instructor
from openai import OpenAI

def agent_node(state: State) -> dict:

   prompt_template =  """You are a assistant that is generating vibes for a user.

You will be given a selection of tools you can use to add vibes to a user's query.

<Available tools>
{{ available_tools | tojson }}
</Available tools>

When you need to use a tool, format your response as:

<tool_call>
{"name": "tool_name", "arguments": {...}}
</tool_call>

Instructions:
- You need to use the tools to add vibes to the user's query.
- Add a random vibe to the user's query.
- Once you have the vibe from the tool, return it as an answer, you can reformulate it a bit.
"""
   # 1. Make a template of the prompt (jinja2 template to inject variables)
   template = Template(prompt_template)
   
   # 2. Inject the available_tools to the prompt
   prompt = template.render(available_tools=state.available_tools)
   
   # 3. NEW: History of messages (state messages converted to open ai json messages format)
   messages = state.messages
   conversation = []
   for message in messages:
      conversation.append(lc_messages_to_regular_messages(message)) # result is a dict{role, content}
   
   
   # 3. Use the instructor to enforce structured output; initialize the client
   client = instructor.from_openai(OpenAI())

   # 4. Use the client to create a response
   # NEW: convert from langchain message types to open ai json messages format
   response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        response_model=AgentResponse,
        messages=[
           {"role": "system", "content": prompt}, 
           *conversation], # NEW: instead of initial message, use history of messages = SHORT TERM MEMORY
        #  [dict{role, content}, dict{role, content}] this is conversation format
        temperature=1.5,
   )

   # 5. Create an ai message (langchain message type)
   # If tool calls, returns LLM response & tools, if no tools - returns response
   # NEW: Add history of messages to the state (messages)
   if response.tool_calls:
      tool_calls = [] # list of tool calls (dicts: id, name, args)
      for i, tc in enumerate(response.tool_calls): 
         tool_calls.append({
               "id": f"call_{i}",
               "name": tc.name,
               "args": tc.arguments
         })
      ai_message = AIMessage( 
         content=response.answer, # LLM response (str)
         tool_calls=tool_calls    # tool_calls (list of dicts)
         )
   else: 
      ai_message = AIMessage( 
         content=response.answer, # LLM response (str)
      )

   return {
      "messages": [ai_message],
      "tool_calls": response.tool_calls, 
      "iteration": state.iteration + 1, # NEW: iteration (number of iterations)
      "answer": response.answer, # NEW: answer (str)
   }
   

In [ ]:
# NEW: additional conditions to run the agent only once
# Tool router: if there are tool calls, go to the tool node, otherwise end the workflow
def tool_router(state: State) -> str:
    """Decide whether to continue or end"""
    if state.iteration > 1: # run only once
        return "end"
    elif len(state.tool_calls) > 0: 
        return "tools"
    else:
        return "end"

In [ ]:
# A graph

workflow = StateGraph(State)

tools = [append_vibes]
tool_node = ToolNode(tools)
tool_descriptions=get_tool_descriptions_from_node(tool_node)

workflow.add_node("agent_node", agent_node)
workflow.add_node("tool_node", tool_node)

workflow.add_edge(START, "agent_node")
workflow.add_conditional_edges(
   "agent_node",
   tool_router,
   {
      "tools": "tool_node", # keys are returned by the tool_router
      "end": END            # keys are returned by the tool_router
   }
)

# This makes the graph ReAct type
workflow.add_edge("tool_node", "agent_node")

graph = workflow.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Execute
# 1. Initial state
initial_state = {
    "messages": [{"role": "user", "content": "Give me some vibes!"}], # we start with messages because we want history
    "available_tools": tool_descriptions
}

# 2. Execute the graph
result = graph.invoke(initial_state)

In [ ]:
result

### **Utils**

In [ ]:
import ast
from typing import Dict, Any

def parse_function_definition(function_def: str) -> Dict[str, Any]:
    """Parse a function definition string to extract metadata including type hints."""
    result = {
        "name": "",
        "description": "",
        "parameters": {"type": "object", "properties": {}},
        "required": [],
        "returns": {"type": "string", "description": ""}
    }
    
    # Parse the function using AST
    tree = ast.parse(function_def.strip())
    if not tree.body or not isinstance(tree.body[0], ast.FunctionDef):
        return result
        
    func = tree.body[0]
    result["name"] = func.name
    
    # Extract docstring
    docstring = ast.get_docstring(func) or ""
    if docstring:
        # Extract description (first line/paragraph)
        desc_end = docstring.find('\n\n') if '\n\n' in docstring else docstring.find('\nArgs:')
        desc_end = desc_end if desc_end > 0 else docstring.find('\nParameters:')
        result["description"] = docstring[:desc_end].strip() if desc_end > 0 else docstring.strip()
        
        # Parse parameter descriptions
        param_descs = parse_docstring_params(docstring)
        
        # Extract return description
        if "Returns:" in docstring:
            result["returns"]["description"] = docstring.split("Returns:")[1].strip().split('\n')[0]
    
    # Extract parameters with type hints
    args = func.args
    defaults = args.defaults
    num_args = len(args.args)
    num_defaults = len(defaults)
    
    for i, arg in enumerate(args.args):
        if arg.arg == 'self':
            continue
            
        param_info = {
            "type": get_type_from_annotation(arg.annotation) if arg.annotation else "string",
            "description": param_descs.get(arg.arg, "")
        }
        
        # Check for default value
        default_idx = i - (num_args - num_defaults)
        if default_idx >= 0:
            param_info["default"] = ast.literal_eval(ast.unparse(defaults[default_idx]))
        else:
            result["required"].append(arg.arg)
        
        result["parameters"]["properties"][arg.arg] = param_info
    
    # Extract return type
    if func.returns:
        result["returns"]["type"] = get_type_from_annotation(func.returns)
    
    return result


def get_type_from_annotation(annotation) -> str:
    """Convert AST annotation to type string."""
    if not annotation:
        return "string"
    
    type_map = {
        'str': 'string',
        'int': 'integer', 
        'float': 'number',
        'bool': 'boolean',
        'list': 'array',
        'dict': 'object',
        'List': 'array',
        'Dict': 'object'
    }
    
    if isinstance(annotation, ast.Name):
        return type_map.get(annotation.id, annotation.id)
    elif isinstance(annotation, ast.Subscript) and isinstance(annotation.value, ast.Name):
        base_type = annotation.value.id
        return type_map.get(base_type, base_type.lower())
    
    return "string"


def parse_docstring_params(docstring: str) -> Dict[str, str]:
    """Extract parameter descriptions from docstring (handles both Args: and Parameters: formats)."""
    params = {}
    lines = docstring.split('\n')
    in_params = False
    current_param = None
    
    for line in lines:
        stripped = line.strip()
        
        # Check for parameter section start
        if stripped in ['Args:', 'Arguments:', 'Parameters:', 'Params:']:
            in_params = True
            current_param = None
        elif stripped.startswith('Returns:') or stripped.startswith('Raises:'):
            in_params = False
        elif in_params:
            # Parse parameter line (handles "param: desc" and "- param: desc" formats)
            if ':' in stripped and (stripped[0].isalpha() or stripped.startswith(('-', '*'))):
                param_name = stripped.lstrip('- *').split(':')[0].strip()
                param_desc = ':'.join(stripped.lstrip('- *').split(':')[1:]).strip()
                params[param_name] = param_desc
                current_param = param_name
            elif current_param and stripped:
                # Continuation of previous parameter description
                params[current_param] += ' ' + stripped
    
    return params

In [ ]:
import inspect
from langgraph.prebuilt import ToolNode

def get_tool_descriptions_from_node(tool_node):
    """Loop all tools and extract tool descriptions from the ToolNode object."""
    descriptions = []
    
    if hasattr(tool_node, 'tools_by_name'):
        tools_by_name = tool_node.tools_by_name
        
        for tool_name, tool in tools_by_name.items():
            function_string = inspect.getsource(globals()[tool_name])
            result = parse_function_definition(function_string)

            if result:
                descriptions.append(result)
    
    return descriptions if descriptions else "Could not extract tool descriptions"

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage
import json

def lc_messages_to_regular_messages(msg):
    """
    Convert various message types to oppen ai compatible format
    """

    if isinstance(msg, dict):
        
        if msg.get("role") == "user":
            return {"role": "user", "content": msg["content"]}
        elif msg.get("role") == "assistant":
            return {"role": "assistant", "content": msg["content"]}
        elif msg.get("role") == "tool":
            return {
                "role": "tool", 
                "content": msg["content"], 
                "tool_call_id": msg.get("tool_call_id")
            }
        
    elif isinstance(msg, AIMessage):

        result = {
            "role": "assistant",
            "content": msg.content
        }
        
        if hasattr(msg, 'tool_calls') and msg.tool_calls and len(msg.tool_calls) > 0 and not msg.tool_calls[0].get("name").startswith("functions."):
            result["tool_calls"] = [
                {
                    "id": tc["id"],
                    "type": "function",
                    "function": {
                        "name": tc["name"].replace("functions.", ""),
                        "arguments": json.dumps(tc["args"])
                    }
                }
                for tc in msg.tool_calls
            ]
            
        return result
    
    elif isinstance(msg, ToolMessage):

        return {"role": "tool", "content": msg.content, "tool_call_id": msg.tool_call_id}
    
    else:

        return {"role": "user", "content": str(msg)}